Optimizing with Delta Engine with Databricks

Delta Engine:
1. Delta Engine is a high performance query engine built into databricks runtime helping to process and query 
2. data faster than the spark engine
3. Great engine for SQL workloads
4. Can work with any data format including delta

Delta Engine components:
1. Query optimizer
  - Data Skipping
  - Bin-packing and Z-Order optimizations
  - Vacuum
  - Auto optimization
  - Data transformation optimization
  - Join and Low shuffle merge optimization
2. Photon execution engine
3. Delta cache


Data Skipping and Statistics

Transaction log entry stores min and max values for each column of the transaction (3 records below)

Select * from employee
where salary between 16000 and 17000

This will check each transaction to see which has min and max salary satisfying this criteria. Id matches then all records in that trnsaction files are queried.
Rest of all files are ignored

{
	"add": {
		"path": "VendorId=3/part-00000-f8957112-b65b-43f0-8713-bd298ae6cf9a.c000.snappy.parquet",
		"partitionValues": {
			"VendorId": "3"
		},
		"size": 5232,
		"modificationTime": 1715496718000,
		"dataChange": true,
		"stats": "{\"numRecords\":3,
    
    \"minValues\":{\"RideId\":9999997,\"PickupTime\":\"2022-03-01T00:00:00.000Z\",\"DropTime\":\"2022-03-01T00:10:56.000Z\",\"PickupLocationId\":141,\"DropLocationId\":68,\"CabNumber\":\"T489328C\",\"DriverLicenseNumber\":\"5067782\",\"PassengerCount\":0,\"TripDistance\":1.1,\"RatecodeId\":1,\"PaymentType\":1,\"TotalAmount\":9.8,\"FareAmount\":8.5,\"Extra\":0.5,\"MtaTax\":0.5,\"TripAmount\":0.0,\"TollsAmount\":0.0,\"ImprovementSurcharge\":0.3,\"PickUpYear\":2022,\"PickUpMonth\":3,\"PickUpDay\":1},
    
    \"maxValues\":{\"RideId\":9999999,\"PickupTime\":\"2022-03-01T00:00:00.000Z\",\"DropTime\":\"2022-03-01T00:15:34.000Z\",\"PickupLocationId\":170,\"DropLocationId\":170,\"CabNumber\":\"TAC399\",\"DriverLicenseNumber\":\"5131685\",\"PassengerCount\":1,\"TripDistance\":2.9,\"RatecodeId\":1,\"PaymentType\":2,\"TotalAmount\":15.3,\"FareAmount\":13.0,\"Extra\":0.5,\"MtaTax\":0.5,\"TripAmount\":2.05,\"TollsAmount\":0.0,\"ImprovementSurcharge\":0.3,\"PickUpYear\":2022,\"PickUpMonth\":3,\"PickUpDay\":1},
    
    \"nullCount\":{\"RideId\":0,\"PickupTime\":0,\"DropTime\":0,\"PickupLocationId\":0,\"DropLocationId\":0,\"CabNumber\":0,\"DriverLicenseNumber\":0,\"PassengerCount\":0,\"TripDistance\":0,\"RatecodeId\":0,\"PaymentType\":0,\"TotalAmount\":0,\"FareAmount\":0,\"Extra\":0,\"MtaTax\":0,\"TripAmount\":0,\"TollsAmount\":0,\"ImprovementSurcharge\":0,\"PickUpYear\":0,\"PickUpMonth\":0,\"PickUpDay\":0}}",
		"tags": {
			"INSERTION_TIME": "1715496718000000",
			"MIN_INSERTION_TIME": "1715496718000000",
			"MAX_INSERTION_TIME": "1715496718000000",
			"OPTIMIZE_TARGET_SIZE": "268435456"
		}
	}
}

In [0]:
%python
# recreate parquet and delta files
dbutils.fs.rm("abfss://datalake@mue10dadls01.dfs.core.windows.net/ShauryaRawat/Output/YellowTaxis.parquet", True)
dbutils.fs.rm("abfss://datalake@mue10dadls01.dfs.core.windows.net/ShauryaRawat/Output/YellowTaxis.delta", True)

In [0]:
%sql
DROP TABLE IF EXISTS taxisDB.YellowTaxisParquet;
DROP TABLE IF EXISTS taxisDB.YellowTaxisDelta

In [0]:
%python
# Create Schema and dataframe 
from pyspark.sql.functions import *
from pyspark.sql.types import *

yellowTaxiSchema = (
    StructType()
   .add("RideId", "integer")
   .add("VendorId", "integer")
   .add("PickupTime", "timestamp")
   .add("DropTime", "timestamp")
   .add("PickupLocationId", "integer")
   .add("DropLocationId", "integer")
   .add("CabNumber", "string")
   .add("DriverLicenseNumber", "string")
   .add("PassengerCount", "integer")
   .add("TripDistance", "double")
   .add("RatecodeId", "integer")
   .add("PaymentType", "integer")
   .add("TotalAmount", "double")
   .add("FareAmount", "double")
   .add("Extra", "double")
   .add("MtaTax", "double")
   .add("TripAmount", "double")
   .add("TollsAmount", "double")
   .add("ImprovementSurcharge", "double")
)
yellowTaxisDF = (
    spark 
    .read 
    .option('header', 'true')
    .schema(yellowTaxiSchema)
    .csv("abfss://datalake@mue10dadls01.dfs.core.windows.net/ShauryaRawat/raw/YellowTaxis/YellowTaxis1.csv")
)
yellowTaxisDF = (
    yellowTaxisDF
    .withColumn("TripYear", year("PickupTime"))
    .withColumn("TripMonth", month("PickupTime"))
    .withColumn("TripDay", dayofmonth("PickupTime"))
)

In [0]:
%python
 # Save table in parquet as well as delta format
 # Data Lake vs Delta Lake

 (
     yellowTaxisDF
     .write
     .mode("overwrite")
     .partitionBy("PickupLocationId")
     .format("parquet")
     .option("path", "abfss://datalake@mue10dadls01.dfs.core.windows.net/ShauryaRawat/Output/YellowTaxis.parquet")
     .saveAsTable("TaxisDB.YellowTaxisParquet")
 )
 (
     yellowTaxisDF
     .write 
     .mode("overwrite")
     .partitionBy("pickupLocationId")
     .format("delta")
     .option("path", "abfss://datalake@mue10dadls01.dfs.core.windows.net/ShauryaRawat/Output/YellowTaxis.delta")
     .saveAsTable("TaxisDB.YellowTaxisDelta")
 )

In [0]:
%sql
-- Took 4.5 seconds
Select count(*) from TaxisDB.YellowTaxisParquet

In [0]:
%sql
-- Took 1.14 seconds
Select count(*) from TaxisDB.YellowTaxisDelta

In [0]:
%sql
Select * from TaxisDB.YellowTaxisParquet
where RideId = 67

Optimizing and Z-Ordering Delta Table

Options:
1. Bin Packing / File compaction
2. Z-Ordering

Bin-Packing ( or File Compaction):
- Convert many small sixe partition files (multiple inserts, updates, delete over time) to bigger size file (evenly balanced, large file, compressed data)
- Run Optimize command on Delta table
- Max size 1 GB by default
- Previous files are also kept to support time travel

Z-Ordering:
- Performs Bin-packing and also sort data based on column (Similar to cluster index on DB)
- Run Optimize ZOrder By(Column)
- Improves performance by data skipping(sorted data is easy to be searched)
- Use column which are frequently used in filetring, joining and grouping

* Both are CPU intensive, costly operation, run command during quite business hours

In [0]:
%sql
-- Check performance before optimization 
-- Data Lake (14.05 s)
SELECT PickupLocationId, Count(*) AS TotalRides
FROM TaxisDB.YellowTaxisParquet
WHERE TripYear = 2022
      AND TripMonth = 3
      AND TripDay = 15
GROUP BY PickupLocationId
ORDER BY TotalRides DESC 

In [0]:
%sql
-- Check performance before optimization  (Data skipping)
-- Delta Lake (6.55 s) 
SELECT PickupLocationId, Count(*) AS TotalRides
FROM TaxisDB.YellowTaxisDelta
WHERE TripYear = 2022
      AND TripMonth = 3
      AND TripDay = 15
GROUP BY PickupLocationId
ORDER BY TotalRides DESC 

In [0]:
%sql
-- Optimize - Bin Packing

OPTIMIZE TaxisDB.YellowTaxisDelta
WHERE PickupLocationId = 100 -- Optional

-- Output
/*numFilesAdded: 1
numFilesRemoved: 10
filesAdded: {"min": 5383836, "max": 5383836, "avg": 5383836, "totalFiles": 1, "totalSize": 5383836}
filesRemoved: {"min": 216600, "max": 755463, "avg": 652716.9, "totalFiles": 10, "totalSize": 6527169}
*/


In [0]:
%sql
-- Optimize: Z Ordering
OPTIMIZE TaxisDB.YellowTaxisDelta
ZORDER BY (TripYear, TripMonth, TripDay)

In [0]:
%sql
-- Check performance after optimization 
-- Delta Lake (1.53 s) 
SELECT PickupLocationId, Count(*) AS TotalRides
FROM TaxisDB.YellowTaxisDelta
WHERE TripYear = 2022
      AND TripMonth = 3
      AND TripDay = 15
GROUP BY PickupLocationId
ORDER BY TotalRides DESC 

VACCUM
1. Physically removes data files
   - Files that are no longer referenced in the latest transaction log state
   - Files older than retention threshold (default is 7)
   - Affects time travel (Set the retention period for time travel and run vaccum periodically)

In [0]:
%sql
DESCRIBE HISTORY TaxisDB.YellowTaxisDelta

In [0]:
%sql
-- Run VACUUM to clean the old files
VACUUM TaxisDB.YellowTaxisDelta
RETAIN 0 HOURS DRY RUN                  -- Default is 7

/* Output
Are you sure you would like to vacuum files with such a low retention period?
Databricks stops us from deleting files with such low retention period
*/

In [0]:
%sql
-- If still want to delete, set retention perios to false
SET spark.databricks.delta.retentionDurationCheck.enabled = false

In [0]:
%sql
-- Run again VACUUM to clean the old files (returns file to be deleted)
VACUUM TaxisDB.YellowTaxisDelta
RETAIN 0 HOURS DRY RUN    

In [0]:
%sql
-- Run again VACUUM to clean the old files (withour dry run)
VACUUM TaxisDB.YellowTaxisDelta
RETAIN 0 HOURS

In [0]:
%sql
-- History is still returned, but files are not available
DESCRIBE HISTORY TaxisDB.YellowTaxisDelta

In [0]:
%sql
-- Older file does not exists
SELECT * FROM TaxisDB.YellowTaxisDelta VERSION AS OF 0

Enabling Auto-Optimization on Delta Table:
1. Optimized Writes (Auto bin-packing while writing)
2. Auto compaction  (Auto bin-packing after writing)

Optimized Writes:
1. Databricks creates multiple spark task to write partition files (small partition files). If Optimized writes are enabled, it creates few other tasks to combine these files into larger file.
2. While writing data, databricks dynamically tries to create ~128 MB partition files
3. Prevents creation of small files without sacrificing parallelism but with slight overhead
4. No need to run optimize command later
5. Use Case: Helpful when loading streaming data and also performing DML operation on table

Auto Compaction:
1. Immediately after writing, databricks further checks partition files in Delta folder
2. Any non-compacted files are compacted to ~128 MB
3. Only runs when there are more than 50 small files
4. No need to run optimize command later
5. Use case: Helpful when loading streaming data and also where optimize command is called after each write

In [0]:
%sql
CREATE TABLE TaxisDB.YellowTaxis_NonOptimized
(
  RideId   INT,
  VendorId INT,
  PickupTime TIMESTAMP,
  DropTime TIMESTAMP,
  PickupLocationId INT,
  DropLocationId INT,
  CabNumber STRING,
  DriverLicenseNumber STRING,
  PassengerCount INT,
  TripDistance DOUBLE,
  RatecodeId INT,
  PaymentType INT,
  TotalAmount DOUBLE,
  FareAmount DOUBLE,
  Extra DOUBLE,
  MtaTax DOUBLE,
  TripAmount DOUBLE,
  TollsAmount DOUBLE,
  ImprovementSurcharge DOUBLE
)
USING DELTA 
LOCATION "abfss://datalake@mue10dadls01.dfs.core.windows.net/ShauryaRawat/Output/YellowTaxis_NonOptimized.delta"
PARTITIONED BY (PickupLocationId)

In [0]:
%python

# Create schema for green taxi data
from pyspark.sql.functions import *
from pyspark.sql.types import *

yellowTaxiSchema = (
    StructType()
    .add("RideId", "integer")
    .add("VendorId", "integer")
    .add("PickupTime", "timestamp")
    .add("DropTime", "timestamp")
    .add("PickupLocationId", "integer")
    .add("DropLocationId", "integer")
    .add("CabNumber", "string")
    .add("DriverLicenseNumber", "string")
    .add("PassengerCount", "integer")
    .add("TripDistance", "double")
    .add("RatecodeId", "integer")
    .add("PaymentType", "integer")
    .add("TotalAmount", "double")
    .add("FareAmount", "double")
    .add("Extra", "double")
    .add("MtaTax", "double")
    .add("TripAmount", "double")
    .add("TollsAmount", "double")
    .add("ImprovementSurcharge", "double")
)

yellowTaxisDF = (
    spark 
    .read 
    .option("header", "true")
    .schema(yellowTaxiSchema)
    .csv("abfss://datalake@mue10dadls01.dfs.core.windows.net/ShauryaRawat/raw/YellowTaxis/YellowTaxis1.csv")
)

In [0]:
%python
(
    yellowTaxisDF 
    .write
    .mode("append")
    .partitionBy("PickupLocationId")
    .format("delta")
    .save("abfss://datalake@mue10dadls01.dfs.core.windows.net/ShauryaRawat/Output/YellowTaxis_NonOptimized.delta")
)

In [0]:
%sql
-- Enabling Auto-Optimized

CREATE TABLE TaxisDB.YellowTaxis_Optimized
(
  RideId   INT                COMMENT 'This is the primary key column',
  VendorId INT,
  PickupTime TIMESTAMP,
  DropTime TIMESTAMP,
  PickupLocationId INT,
  DropLocationId INT,
  CabNumber STRING,
  DriverLicenseNumber STRING,
  PassengerCount INT,
  TripDistance DOUBLE,
  RatecodeId INT,
  PaymentType INT,
  TotalAmount DOUBLE,
  FareAmount DOUBLE,
  Extra DOUBLE,
  MtaTax DOUBLE,
  TripAmount DOUBLE,
  TollsAmount DOUBLE,
  ImprovementSurcharge DOUBLE
)
USING DELTA 
LOCATION "abfss://datalake@mue10dadls01.dfs.core.windows.net/ShauryaRawat/Output/YellowTaxis_Optimized.delta"
PARTITIONED BY (PickupLocationId)

TBLPROPERTIES (delta.autoOptimize.optimizeWrite = true, delta.autoOptimize.autoCompact = true)

In [0]:
%sql
-- Auto-Optimized created one file instead of 10 files in each partition
%python
(
    yellowTaxisDF
    .write
    .mode("append")
    .partitionBy("PickupLocationId")
    .format("delta")
    .save("abfss://datalake@mue10dadls01.dfs.core.windows.net/ShauryaRawat/Output/YellowTaxis_Optimized.delta")
)

Photon Execution Engine

1. Native vectorized query engine built by databricks
2. Written in C++ taking benefit of modern hardware
3. Works with Delta and Parquet based tables
4. Much faster than spark against high data volumes
5. To use it, select databricks runtime with photon
6. Different pricing than spark based clusters


Now, Photon is supported by default in clusters 
New -> Compute (Cluster)
Option: Use Photon Acceleration above 9.1 LTS runtime

Caching Types

1. Apache Spark Cache (Part of Spark)
2. Delta Cache (Part of Delta Engine)

How Delta Cache Works?
1. Works for Parquet and Delta formats
2. Files are read from external storage for the first time
3. Stored on local disks of cluster machines
4. Successive reads of same data are served using local files, improving query performance 
5. Cache is automatically invalidated / evicted if:
   - Underlying files are modified
   - Cluster is restarted

How to use Delta Cache?
- While creating cluster, select "Use Delta Cache accelerated VMs" as worker machines - caching enabled by default
  - When creating cluster 2 options exists for worker types:
    1. Memory optimized (Delta cache accelerated)
    2. Storage optimized (Delta cache accelerated)
- For other type of machines (VM), explicitly enable the cache

In [0]:
%python
# Check if delta cache is enabled
spark.conf.get("spark.databricks.io.cache.enabled")


'true'

In [0]:
%python
# To enable the cache (if not enabled) use 
spark.conf.set("spark.databricks.io.cache.enabled", "true")

In [0]:
%sql
-- If delta cache is not provided in Cluster, you can explicitly push data to delta cache, with CACHE keyword

CACHE
SELECT * from TaxisDB.YellowTaxisDelta
WHERE TripDay = 1

Delta Engine - Summary
1. High performance query engine built into runtime
2. 3 components: Optimizations, Photon, Delta Cache